In [2]:
from database.models import HistoricalOHLC
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import tools.tools as tools
import requests
import json

### Inicialização do Banco SQLite

In [3]:
historical = HistoricalOHLC()
historical.drop_table()
historical.create_table()

### Extração de Dados da API OHLC (Open, High, Low, Close) e Inserção no Banco

In [4]:
#Configs
today = date.today()
dt_format = "%Y-%m-%d"

BASE_OHLC_URL = 'https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical'

for i in range(0,60):
    
    date_ini = today - relativedelta(months=i+1)
    date_end = today - relativedelta(months=i)
    start_unixtime = tools.to_unixtime_sec(date_ini.strftime(dt_format))
    end_unixtime = tools.to_unixtime_sec(date_end.strftime(dt_format))

    #Extração 
    param = f'?id=1&convertId=2783&timeStart={start_unixtime}&timeEnd={end_unixtime}&interval=1d'
    response = requests.get(BASE_OHLC_URL + param)
    items = response.json()['data']['quotes']

    for item in items:
        #Inserção
        historical.insert((
            item['timeOpen'][0:10],
            item['quote']['open'],
            item['quote']['high'],
            item['quote']['low'],
            item['quote']['close']
        ))


### Extração dos Dados Formatados do Banco de Dados

In [11]:
today = date.today()
dt_format = "%Y-%m-%d"

date_1month = today - relativedelta(months=1)
date_5year = today - relativedelta(years=5)

graph_candle = historical.read_with_filter(f'date between date("{date_1month.strftime(dt_format)}") and date("{today.strftime(dt_format)}")')
graph_all = historical.read_with_filter(f'date between date("{date_5year.strftime(dt_format)}") and date("{today.strftime(dt_format)}")')


### Gráficos